In [ ]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

In [ ]:
import os
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kubernetes import client as k8s_client

In [ ]:
dkube_training_op           = kfp.components.load_component_from_url("https://raw.githubusercontent.com/oneconvergence/dkube-examples/citiustech/components/training/component.yaml")
dkube_serving_op            = kfp.components.load_component_from_url("https://raw.githubusercontent.com/oneconvergence/dkube-examples/citiustech/components/serving/component.yaml")

In [ ]:
image = "ocdr/detectron2:gpu"
serving_image = "nvcr.io/nvidia/tritonserver:20.06-py3"
dataset = 'baloon'
training_program = 'detectron2'
model = 'detectron2'
training_script = "python training.py"
training_gpus = 1
user = os.getenv('USERNAME')
framework = "custom"
output_mount_point = "/opt/dkube/output/"
input_mount_point = "/opt/dkube/input/"

In [ ]:
@kfp.dsl.pipeline(
    name='dkube-detectron-pl',
    description='sample detectron example pipeline'
)
def detectron_pipeline(token):

    train       = dkube_training_op(token, json.dumps({"image": image}),
                                    framework=framework, version=f_version,
                                    program=training_program, run_script=training_script,
                                    datasets=json.dumps([dataset]), outputs=json.dumps([model]),
                                    input_dataset_mounts=json.dumps([input_mount_point]),
                                    ngpus=training_gpus,
                                    output_mounts=json.dumps([output_mount_point]))

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": serving_image}),
                                    transformer_image=json.dumps({"image": image}),
                                    transformer_project=training_program).after(train)

# Create a run

In [ ]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
client.create_run_from_pipeline_func(detectron_pipeline, arguments={"token":existing_token})

# Compile and generate tar ball

In [ ]:
compiler.Compiler().compile(detectron_pipeline, 'dkube_detectron_pl.tar.gz')

# Upload pipeline

In [ ]:
client = kfp.Client(existing_token=existing_token)
try:
    client.upload_pipeline(pipeline_package_path = 'dkube_detectron_pl.tar.gz', pipeline_name = 'Detectron pipeline', description = None)
except BaseException as e:
    print(e)